In [1]:
'''
task: extract ontology terms and types from documents
model: gemma-2-2b-it
dataset: scholarly
evaluation: zero-shot with prompting
'''
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
# read data

import json

with open("/content/drive/MyDrive/Colab Notebooks/LLMs4OL/subtaskA/data/scholarly/train/documents.jsonl", "r") as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    print(f"result: {result}")
    print(isinstance(result, dict))
    print(result["text"])

result: {'id': '36_0', 'title': 'Temporal Qualifiers in Linguistics: Outdated, Obsolete, and Archaic Forms', 'text': 'In linguistics, certain forms are used to indicate that a word or expression is no longer in current use. These forms are classified as temporal qualifiers, which provide context about the timing or period when a particular linguistic expression was used. Specifically, the outdated form, obsolete form, and archaic form are all types of temporal qualifiers. The outdated form refers to a word or expression that was once commonly used but has since fallen out of favor. Similarly, the obsolete form is a word or expression that has completely gone out of use. The archaic form, on the other hand, is a word or expression that is no longer used in everyday language but may still be found in historical or literary contexts. All these forms serve as temporal qualifiers, helping to situate the language in a particular time or context.'}
True
In linguistics, certain forms are used 

In [2]:
# start preparing for QA pipeline
! pip install -U accelerate
! pip install -U transformers
!pip install transformers
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [3]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM

import warnings
warnings.filterwarnings("ignore")

In [4]:
# login to hugging face to have access to the model
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

In [5]:
# load zero-shot model
MODEL_NAME = "google/gemma-2-2b-it"
TOKENIZER = AutoTokenizer.from_pretrained(MODEL_NAME)
MODEL = AutoModelForCausalLM.from_pretrained(MODEL_NAME, return_dict=True)
DEVICE = "cuda:0"

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [6]:
#torch.cuda.empty_cache()
MODEL.to('cuda')

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemm

In [136]:
# evaluation metrics

import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import re

def precision_at_k(actual, predicted):
    act_set = set(actual)
    pred_set = set(predicted)
    result = len(act_set & pred_set) / float(len(predicted))
    return result*100

def apk(actual, predicted, k):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if not actual:
        return 0.0
    if len(predicted)>k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    return score / min(len(actual), k)


def mapk(actual, predicted, k):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int,
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a, p, k) for a,p in zip(actual, predicted)])


class EvaluationMetrics:

    def __init__(self, ks:list, metric="map") -> None:
        self.ks = ks
        self.metric=metric

    def evaluate(self, actual:list, predicted:list):
        if self.metric == "map":
            return self.MAP(actual, predicted)
        else:
            return self.AP(actual, predicted)

    def MAP(self, actual:list, predicted:list):
        results_dict = {}
        for k in self.ks:
            results_dict["MAP@"+str(k)] = mapk(actual=actual, predicted=predicted, k=k)
        return results_dict

    def AP(self, actual:list, predicted:list):
        results_dict = {}
        for k in self.ks:
            results_dict["AP@"+str(k)] = [apk(actual=actual, predicted=predicted, k=k)
                                          for a, p in zip(actual, predicted)]
        return results_dict

def predict_answer(document):
    prompt = f"""
    <start_of_turn>user
    You are an expert ontologist. You are given a text between <INPUT></INPUT> tags and you need to extract all terms and types to create an ontology. A type is to a class what a term is to an object in object-oriented definition. Limit your output to 2 lists with keys 'terms' and 'types' and the values for each the list of terms and types respectively between <output> tags.
	  <INPUT>{document}</INPUT>
    <end_of_turn>
    <start_of_turn>model
    """
    inputs = TOKENIZER(prompt, return_tensors="pt").to(DEVICE)

    outputs = MODEL.generate(**inputs, max_new_tokens=120)

    predicted_relation = TOKENIZER.decode(outputs[0], skip_special_tokens=True)

    result = re.search(r'\{([^}]*)\}', predicted_relation)  # locate json object
    json_str =  result.group(1) if result else "None"
    json_answer = "{"+json_str.replace("\'", "\"")+"}" if json_str else "None"

    print("document: \n", document)
    print("predicted answer: \n", predicted_relation)
    return json.loads(json_answer) if json_answer else "None"

In [137]:
# test on one example

document = "In linguistics, certain forms are used to indicate that a word or expression is no longer in current use. These forms are classified as temporal qualifiers, which provide context about the timing or period when a particular linguistic expression was used. Specifically, the outdated form, obsolete form, and archaic form are all types of temporal qualifiers. The outdated form refers to a word or expression that was once commonly used but has since fallen out of favor. Similarly, the obsolete form is a word or expression that has completely gone out of use. The archaic form, on the other hand, is a word or expression that is no longer used in everyday language but may still be found in historical or literary contexts. All these forms serve as temporal qualifiers, helping to situate the language in a particular time or context."
predicted_answer = predict_answer(document)
predicted_answer

document: 
 In linguistics, certain forms are used to indicate that a word or expression is no longer in current use. These forms are classified as temporal qualifiers, which provide context about the timing or period when a particular linguistic expression was used. Specifically, the outdated form, obsolete form, and archaic form are all types of temporal qualifiers. The outdated form refers to a word or expression that was once commonly used but has since fallen out of favor. Similarly, the obsolete form is a word or expression that has completely gone out of use. The archaic form, on the other hand, is a word or expression that is no longer used in everyday language but may still be found in historical or literary contexts. All these forms serve as temporal qualifiers, helping to situate the language in a particular time or context.
predicted answer: 
 
    user
    You are an expert ontologist. You are given a text between <INPUT></INPUT> tags and you need to extract all terms and 

{'terms': ['outdated form', 'obsolete form', 'archaic form'],
 'types': ['temporal qualifier', 'temporal qualifier', 'temporal qualifier']}

In [104]:
# predict on all documents
for json_str in json_list:
    result = json.loads(json_str)
    predicted_answer = predict_answer(result["text"])

document: 
 In linguistics, certain forms are used to indicate that a word or expression is no longer in current use. These forms are classified as temporal qualifiers, which provide context about the timing or period when a particular linguistic expression was used. Specifically, the outdated form, obsolete form, and archaic form are all types of temporal qualifiers. The outdated form refers to a word or expression that was once commonly used but has since fallen out of favor. Similarly, the obsolete form is a word or expression that has completely gone out of use. The archaic form, on the other hand, is a word or expression that is no longer used in everyday language but may still be found in historical or literary contexts. All these forms serve as temporal qualifiers, helping to situate the language in a particular time or context.
predicted answer: 
 
    user
    You are an expert ontologist. You are given a text between <INPUT></INPUT> tags and you need to extract all terms and 

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)